In [1]:
## Carga de Contrato
import pandas as pd
import numpy as np
from datetime import datetime
import os
import sys
from io import StringIO

# Configuração dos caminhos dos arquivos
planilha_destino = r"C:\Users\lz6321\Downloads\carga_dados_contratos_sofi.xlsx"  # Arquivo a ser preenchido
planilha_fonte1 = r"C:\Users\lz6321\Downloads\contratos 2025.xlsm"   # Primeira fonte de dados
planilha_fonte2 = r"C:\Users\lz6321\Downloads\TI - Previsão Gastos 2025 - Contratos\Cópia de 1 -Comunicação de Dados TI 2025.xlsx"   # Segunda fonte de dados

def converter_para_float(valor_bruto):
    try:
        # Remove os caracteres não numéricos, mantendo pontos e vírgulas
        valor_limpo = ''.join(c for c in str(valor_bruto) if c.isdigit() or c in ',.').strip()
        if not valor_limpo:
            return '0,00'

        # Normaliza o formato do número
        if ',' in valor_limpo:
            valor_limpo = valor_limpo.replace('.', '').replace(',', '.')
        
        # Converte para float
        valor_float = float(valor_limpo)
        
        # Formata o número com pontos para milhares e vírgula para decimais
        return f'{valor_float:,.2f}'.replace(',', 'X').replace('.', ',').replace('X', '.')
    except (ValueError, AttributeError):
        return '0,00'

def analisar_planilhas():
    # Criar um buffer para capturar a saída
    output = StringIO()
    stdout_original = sys.stdout
    sys.stdout = output
    
    try:
        # Analisando planilha destino
        print('\n=== Análise da Planilha Destino ===')
        df_destino = pd.read_excel(planilha_destino)
        print(f'Número de linhas: {df_destino.shape[0]}')
        print(f'Número de colunas: {df_destino.shape[1]}')
        print('\nColunas disponíveis:')
        for idx, coluna in enumerate(df_destino.columns, 1):
            print(f'{idx}. {coluna}')
        print('\nPrimeiras 5 linhas da planilha destino:')
        print(df_destino.head().to_string())
        
        # Analisando primeira aba da planilha fonte 1
        print('\n=== Análise da Planilha Fonte 1 (Primeira Aba) ===')
        df_fonte1 = pd.read_excel(planilha_fonte1, sheet_name=0)
        print(f'Número de linhas: {df_fonte1.shape[0]}')
        print(f'Número de colunas: {df_fonte1.shape[1]}')
        print('\nColunas disponíveis:')
        for idx, coluna in enumerate(df_fonte1.columns, 1):
            print(f'{idx}. {coluna}')
        print('\nPrimeiras 5 linhas da planilha fonte 1:')
        print(df_fonte1.head().to_string())
        
        # Analisando abas válidas da planilha fonte 2
        print('\n=== Análise da Planilha Fonte 2 ===')
        todas_abas_fonte2 = pd.read_excel(planilha_fonte2, sheet_name=None)
        
        # Removendo as abas que devem ser ignoradas
        abas_ignorar = ['CC', 'Resumo']
        abas_validas = {nome: df for nome, df in todas_abas_fonte2.items() if nome not in abas_ignorar}
        
        print(f'Número de abas válidas: {len(abas_validas)}')
        print('\nAnálise de cada aba válida:')
        
        for nome_aba, df in abas_validas.items():
            print(f'\n--- Aba: {nome_aba} ---')
            print(f'Número de linhas: {df.shape[0]}')
            print(f'Número de colunas: {df.shape[1]}')
            print('Colunas disponíveis:')
            for idx, coluna in enumerate(df.columns, 1):
                print(f'{idx}. {coluna}')
            print('\nPrimeiras 2 linhas desta aba:')
            print(df.head(2).to_string())
            print('-' * 50)
        
        # Restaurar stdout original
        sys.stdout = stdout_original
        
        # Salvar o resultado em um arquivo txt
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        caminho_txt = os.path.join(r"C:\Users\lz6321\Downloads", f'analise_planilhas_{timestamp}.txt')
        
        with open(caminho_txt, 'w', encoding='utf-8') as f:
            f.write(output.getvalue())
        
        print(f'✅ Arquivo de análise salvo em: {caminho_txt}')
        
        return df_destino, df_fonte1, abas_validas
        
    except Exception as e:
        # Restaurar stdout em caso de erro
        sys.stdout = stdout_original
        print(f'❌ Erro ao analisar planilhas: {str(e)}')
        return None, None, None
    finally:
        # Garantir que stdout seja restaurado
        sys.stdout = stdout_original

def formatar_data(data_str):
    """Converte uma data para o formato dd/mm/yyyy"""
    try:
        if not data_str or pd.isna(data_str):
            return ''
        
        # Converte para datetime
        if isinstance(data_str, str):
            try:
                data = pd.to_datetime(data_str, format='%d/%m/%Y')
            except:
                data = pd.to_datetime(data_str)
        else:
            data = pd.to_datetime(data_str)
            
        # Formata para dd/mm/yyyy
        return data.strftime('%d/%m/%Y')
    except Exception as e:
        print(f"Erro ao formatar data '{data_str}': {str(e)}")
        return ''

def processar_dados(df_destino, df_fonte1, abas_fonte2):
    # Colunas que queremos manter no arquivo final
    colunas_desejadas = [
        'nome_contrato', 'nome_fornecedor', 'conta_contabil', 'status',
        'vigencia', 'data_inicial', 'data_final', 'valor_total', 'responsavel',
        'vertical', 'descricao', 'despesa_antecipada', 'frequencia', 'observacoes',
        'descricao_rateio'
    ]
    
    # Criar um DataFrame vazio apenas com as colunas desejadas
    df_final = pd.DataFrame(columns=colunas_desejadas)
    
    # Converter nomes das colunas para minúsculo para facilitar a comparação
    df_fonte1.columns = df_fonte1.columns.str.lower()
    
    print("\nProcessando planilha fonte 1:")
    print("\nColunas disponíveis:")
    print(df_fonte1.columns.tolist())
    
    # Mapeia as colunas da planilha fonte 1 para as colunas desejadas
    for i, row in df_fonte1.iterrows():
        try:
            # Extrai os dados das colunas, usando índice 5 (coluna F) para nome_contrato e nome_fornecedor
            nome_contrato = str(row.iloc[5])
            nome_fornecedor = str(row.iloc[5])  # Mesmo valor que nome_contrato
            
            # Extrai outros dados
            valor_total = converter_para_float(row.get('valor', 0))
            frequencia = str(row.get('frequência', ''))
            
            # Pega as datas e formata
            data_inicial = formatar_data(row.get('inicio', ''))
            data_final = formatar_data(row.get('termino', ''))
            
            conta_contabil = str(row.get('conta contabil', ''))
            descricao = str(row.get('descrição', ''))
            despesa_antecipada = str(row.get('despesa antecipada', ''))
            
            # Pega o responsável diretamente da coluna I (indice 8)
            responsavel = str(row.iloc[8])
            
            # Pega observações da coluna H (índice 7)
            observacoes = str(row.iloc[7])
            
            # Pega descrição de rateio da coluna O (índice 14)
            descricao_rateio = str(row.iloc[14])
            
            print(f"\nProcessando linha {i}:")
            print(f"Fornecedor: {nome_fornecedor}")
            print(f"Valor Total: {valor_total}")
            print(f"Data Inicial: {data_inicial}")
            print(f"Data Final: {data_final}")
            print(f"Conta Contabil: {conta_contabil}")
            print(f"Responsável (da coluna I): {responsavel}")
            print(f"Observações (da coluna H): {observacoes}")
            print(f"Descrição Rateio (da coluna O): {descricao_rateio}")
            print(f"Despesa Antecipada: {despesa_antecipada}")
            
            nova_linha = {
                'nome_contrato': nome_contrato,
                'nome_fornecedor': nome_fornecedor,
                'conta_contabil': conta_contabil,
                'status': 'ATIVO',
                'vigencia': '',  # Agora deixa vigencia vazia
                'data_inicial': data_inicial if data_inicial else '',
                'data_final': data_final if data_final else '',
                'valor_total': valor_total,
                'responsavel': responsavel,
                'vertical': '',  # Agora deixa vertical vazia
                'descricao': descricao if descricao else f'Contrato de serviço {nome_contrato}',
                'despesa_antecipada': despesa_antecipada,
                'frequencia': frequencia if frequencia else 'MENSAL',
                'observacoes': observacoes,  # Adiciona observações da coluna H
                'descricao_rateio': descricao_rateio  # Adiciona descrição de rateio da coluna O
            }
            
            df_final = pd.concat([df_final, pd.DataFrame([nova_linha])], ignore_index=True)
            
        except Exception as e:
            print(f"Erro ao processar linha {i}: {str(e)}")
            continue
    
    # Salvar o resultado em um novo arquivo Excel
    caminho_saida = os.path.join(r'C:\Users\lz6321\Downloads', 'contratos_processados.xlsx')
    df_final.to_excel(caminho_saida, index=False)
    print(f'\n✅ Arquivo processado salvo em: {caminho_saida}')
    
    print('\nPrimeiras linhas do arquivo gerado:')
    print(df_final.head())
    
    return df_final

# Execute estas linhas para analisar as planilhas
df_destino, df_fonte1, abas_fonte2 = analisar_planilhas()

# Processar os dados e gerar o arquivo final
df_final = processar_dados(df_destino, df_fonte1, abas_fonte2)
print('\nPrimeiras linhas do arquivo gerado:')
display(df_final.head())

✅ Arquivo de análise salvo em: C:\Users\lz6321\Downloads\analise_planilhas_20250611_161437.txt

Processando planilha fonte 1:

Colunas disponíveis:
['1-5', 'tema', 'inflação', 'prev', 'gov', 'fornecedor', 'descrição', 'serviço/software/sistema', 'responsável ', 'valor', 'frequência', 'inicio', 'termino', 'despesa antecipada', 'rateio ', 'conta contabil', 'ultima req', 'req antiga', 'diversos', 'juridico', 'atualizado', 'notificado', 'unnamed: 22', 'unnamed: 23']

Processando linha 0:
Fornecedor: Locaweb
Valor Total: 90.000,00
Data Inicial: 
Data Final: 19/10/2023
Conta Contabil: Comunicacao de Dados - TI
Responsável (da coluna I): Andre Felipe Grehs
Observações (da coluna H): Hospedagem Servidores + registro domínios nacionais e Internacionais
Descrição Rateio (da coluna O): 34% MKT 33% DVE  33% EXP
Despesa Antecipada: Sim e não

Processando linha 1:
Fornecedor: FAPESP / NIC.BR
Valor Total: 3.900,00
Data Inicial: 
Data Final: 
Conta Contabil: Comunicacao de Dados - TI
Responsável (da c

C:\Users\lz6321\AppData\Local\Temp\ipykernel_15796\999940445.py:118: UserWarning: Parsing dates in %d/%m/%Y	 format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data = pd.to_datetime(data_str)



✅ Arquivo processado salvo em: C:\Users\lz6321\Downloads\contratos_processados.xlsx

Primeiras linhas do arquivo gerado:
              nome_contrato           nome_fornecedor  \
0                   Locaweb                   Locaweb   
1           FAPESP / NIC.BR           FAPESP / NIC.BR   
2               101 Domains               101 Domains   
3    INTERNET LOJA RIDER SP    INTERNET LOJA RIDER SP   
4  INTERNET GALERIA MELISSA  INTERNET GALERIA MELISSA   

              conta_contabil status vigencia data_inicial  data_final  \
0  Comunicacao de Dados - TI  ATIVO                        19/10/2023   
1  Comunicacao de Dados - TI  ATIVO                                     
2  Comunicacao de Dados - TI  ATIVO                                     
3  Comunicacao de Dados - TI  ATIVO                                     
4  Comunicacao de Dados - TI  ATIVO                                     

  valor_total         responsavel vertical  \
0   90.000,00  Andre Felipe Grehs            
1   

,nome_contrato,nome_fornecedor,conta_contabil,status,vigencia,data_inicial,data_final,valor_total,responsavel,vertical,descricao,despesa_antecipada,frequencia,observacoes,descricao_rateio
0,Locaweb,Locaweb,Comunicacao de Dados - TI,ATIVO,,,19/10/2023,"90.000,00",Andre Felipe Grehs,,Sites e Domínios de internet das áreas,Sim e não,Anual,Hospedagem Servidores + registro domínios naci...,34% MKT 33% DVE 33% EXP
1,FAPESP / NIC.BR,FAPESP / NIC.BR,Comunicacao de Dados - TI,ATIVO,,,,"3.900,00",Andre Felipe Grehs,,Sites e Domínios de internet das áreas,Não,Pontualmente,Hospedagem Servidores + registro domínios naci...,Conforme necessidade do setor se vai querer re...
2,101 Domains,101 Domains,Comunicacao de Dados - TI,ATIVO,,,,"2.912,00",Andre Felipe Grehs,,Sites e Domínios de internet das áreas,Não,Pontualmente,Domínios internacionais da Melissa,Melissa
3,INTERNET LOJA RIDER SP,INTERNET LOJA RIDER SP,Comunicacao de Dados - TI,ATIVO,,,,"1.980,00",Andre Felipe Grehs,,Contrato de Internet para loja/unidade,Não,Anual,LINK DE INTERNET (CLARO e VIVO),Loja Rider
4,INTERNET GALERIA MELISSA,INTERNET GALERIA MELISSA,Comunicacao de Dados - TI,ATIVO,,,,"18.000,00",Andre Felipe Grehs,,Contrato de Internet para loja/unidade,Não,Anual,LINK DE INTERNET (CLARO e VIVO),Galeria melissa
